EE798 paper 2 submission

In [17]:
import os
import numpy as np
import pandas as pd
import scipy.io
from scipy.stats import skew, kurtosis
from scipy.signal import welch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy
from numpy import log, isinf



In [18]:

def higuchi_fd(signal, kmax=20):
    
    L = np.array([])
    N = len(signal)
    
    for k in range(1, kmax + 1):
        m = np.floor(N / k).astype(int)
        L_k = 0
        
        for j in range(k):
            segment = signal[j:m * k:k]
            L_k += np.sum(np.abs(np.diff(segment))) 
        
        if m > 0: 
            L_k /= m  
            L = np.append(L, L_k)

    logL = np.log(L)
    logk = np.log(np.arange(1, kmax + 1))

    
    valid_mask = np.isfinite(logL) & np.isfinite(logk)
    logL_filtered = logL[valid_mask]
    logk_filtered = logk[valid_mask]
    
    
    print(f"logL_filtered shape: {logL_filtered.shape}, logk_filtered shape: {logk_filtered.shape}")  # Debug print
    if len(logL_filtered) == len(logk_filtered) and len(logL_filtered) > 0:
        coeffs = np.polyfit(logk_filtered, logL_filtered, 1)
        return coeffs[0]
    else:
        print("Warning: logL_filtered and logk_filtered lengths do not match.")
        return None  




def cross_correlation(x, y):
    correlation = np.correlate(x, y, mode='full')
    return np.max(correlation)  



def hurst_exponent(ts):
    lags = range(2, 100)  
    tau = [np.std(np.subtract(ts[lag:], ts[:-lag])) for lag in lags]
    log_lags = np.log(lags)
    log_tau = np.log(tau)
    return np.polyfit(log_lags, log_tau, 1)[0]  




# Custom spectral entropy function
def spectral_entropy(data, fs=256):
    freqs, psd = welch(data, fs=fs)
    psd_norm = psd / np.sum(psd) 
    entropy = -np.sum(psd_norm * np.log2(psd_norm + 1e-10))
    return entropy

def extract_features(eeg_data):
    features = {}
    
    # Low-level features
    features['mean'] = list(np.mean(eeg_data, axis=1))
    features['variance'] = list(np.var(eeg_data, axis=1))
    features['skewness'] = list(skew(eeg_data, axis=1))
    features['kurtosis'] = list(kurtosis(eeg_data, axis=1))
    features['energy'] = list(np.sum(np.square(eeg_data), axis=1))
    features['rms'] = list(np.sqrt(np.mean(np.square(eeg_data), axis=1)))

    # Mid-level features
    freqs, psd = welch(eeg_data, fs=256, axis=1)  # Adjust fs as per EEG data's sampling rate
    features['band_power_delta'] = list(np.sum(psd[:, (freqs >= 0.5) & (freqs < 4)], axis=1))
    features['band_power_theta'] = list(np.sum(psd[:, (freqs >= 4) & (freqs < 8)], axis=1))
    features['spectral_entropy'] = [spectral_entropy(eeg_data[i]) for i in range(eeg_data.shape[0])]
    
    # High-level features
    features['fractal_dimension'] = [higuchi_fd(eeg_data[i]) for i in range(eeg_data.shape[0])]
    
    if eeg_data.shape[0] >= 2:  # Check if we have at least 2 channels
        for i in range(eeg_data.shape[0]):
            channel_1 = eeg_data[i, 0].ravel()  
            channel_2 = eeg_data[i, 1].ravel() 
            print(f"Cross-correlation input shapes for {i}: {channel_1.shape}, {channel_2.shape}") 
            features.setdefault('cross_correlation', []).append(cross_correlation(channel_1, channel_2))
    else:
        features['cross_correlation'] = [None] * eeg_data.shape[0]

    features['hurst_exponent'] = [hurst_exponent(eeg_data[i]) for i in range(eeg_data.shape[0])]
    
    flattened_features = {f'{key}_{i}': values[i] for key, values in features.items() for i in range(len(values))}
    return flattened_features




main_dir = r'EEGData\EEGData'

data_rows = []

for participant_folder in os.listdir(main_dir):
    participant_path = os.path.join(main_dir, participant_folder)
    if os.path.isdir(participant_path):
        for clip_file in os.listdir(participant_path):
            if clip_file.endswith('.mat'):
                file_path = os.path.join(participant_path, clip_file)
                
                mat_data = scipy.io.loadmat(file_path)
                eeg_data = mat_data['ThisEEG']  
                
                
                features = extract_features(eeg_data)
                
                
                features['participant'] = participant_folder
                features['clip'] = clip_file
                
                
                data_rows.append(features)

#feature extracted dataframe
df = pd.DataFrame(data_rows)


logL_filtered shape: (20,), logk_filtered shape: (20,)
logL_filtered shape: (20,), logk_filtered shape: (20,)
logL_filtered shape: (20,), logk_filtered shape: (20,)
logL_filtered shape: (20,), logk_filtered shape: (20,)
logL_filtered shape: (20,), logk_filtered shape: (20,)
logL_filtered shape: (20,), logk_filtered shape: (20,)
logL_filtered shape: (20,), logk_filtered shape: (20,)
logL_filtered shape: (20,), logk_filtered shape: (20,)
Cross-correlation input shapes for 0: (1,), (1,)
Cross-correlation input shapes for 1: (1,), (1,)
Cross-correlation input shapes for 2: (1,), (1,)
Cross-correlation input shapes for 3: (1,), (1,)
Cross-correlation input shapes for 4: (1,), (1,)
Cross-correlation input shapes for 5: (1,), (1,)
Cross-correlation input shapes for 6: (1,), (1,)
Cross-correlation input shapes for 7: (1,), (1,)
logL_filtered shape: (20,), logk_filtered shape: (20,)
logL_filtered shape: (20,), logk_filtered shape: (20,)
logL_filtered shape: (20,), logk_filtered shape: (20,)
log

C:\Users\amols\AppData\Local\Temp\ipykernel_21884\170670477.py:67: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  features['skewness'] = list(skew(eeg_data, axis=1))
C:\Users\amols\AppData\Local\Temp\ipykernel_21884\170670477.py:68: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  features['kurtosis'] = list(kurtosis(eeg_data, axis=1))
C:\Users\amols\AppData\Local\Temp\ipykernel_21884\170670477.py:57: RuntimeWarning: invalid value encountered in divide
  psd_norm = psd / np.sum(psd)
C:\Users\amols\AppData\Local\Temp\ipykernel_21884\170670477.py:18: RuntimeWarning: divide by zero encountered in log
  logL = np.log(L)
C:\Users\amols\AppData\Local\Temp\ipykernel_21884\170670477.py:48: RuntimeWarning: divide by zero encountered in log
  log_tau

logL_filtered shape: (20,), logk_filtered shape: (20,)
logL_filtered shape: (20,), logk_filtered shape: (20,)
logL_filtered shape: (20,), logk_filtered shape: (20,)
logL_filtered shape: (20,), logk_filtered shape: (20,)
logL_filtered shape: (20,), logk_filtered shape: (20,)
logL_filtered shape: (20,), logk_filtered shape: (20,)
logL_filtered shape: (20,), logk_filtered shape: (20,)
logL_filtered shape: (20,), logk_filtered shape: (20,)
Cross-correlation input shapes for 0: (1,), (1,)
Cross-correlation input shapes for 1: (1,), (1,)
Cross-correlation input shapes for 2: (1,), (1,)
Cross-correlation input shapes for 3: (1,), (1,)
Cross-correlation input shapes for 4: (1,), (1,)
Cross-correlation input shapes for 5: (1,), (1,)
Cross-correlation input shapes for 6: (1,), (1,)
Cross-correlation input shapes for 7: (1,), (1,)
logL_filtered shape: (20,), logk_filtered shape: (20,)
logL_filtered shape: (0,), logk_filtered shape: (0,)
logL_filtered shape: (0,), logk_filtered shape: (0,)
logL_fi

C:\Users\amols\AppData\Local\Temp\ipykernel_21884\170670477.py:70: RuntimeWarning: invalid value encountered in sqrt
  features['rms'] = list(np.sqrt(np.mean(np.square(eeg_data), axis=1)))


logL_filtered shape: (0,), logk_filtered shape: (0,)
logL_filtered shape: (0,), logk_filtered shape: (0,)
logL_filtered shape: (0,), logk_filtered shape: (0,)
logL_filtered shape: (0,), logk_filtered shape: (0,)
logL_filtered shape: (0,), logk_filtered shape: (0,)
logL_filtered shape: (0,), logk_filtered shape: (0,)
logL_filtered shape: (0,), logk_filtered shape: (0,)
logL_filtered shape: (0,), logk_filtered shape: (0,)
Cross-correlation input shapes for 0: (1,), (1,)
Cross-correlation input shapes for 1: (1,), (1,)
Cross-correlation input shapes for 2: (1,), (1,)
Cross-correlation input shapes for 3: (1,), (1,)
Cross-correlation input shapes for 4: (1,), (1,)
Cross-correlation input shapes for 5: (1,), (1,)
Cross-correlation input shapes for 6: (1,), (1,)
Cross-correlation input shapes for 7: (1,), (1,)
logL_filtered shape: (0,), logk_filtered shape: (0,)
logL_filtered shape: (0,), logk_filtered shape: (0,)
logL_filtered shape: (0,), logk_filtered shape: (0,)
logL_filtered shape: (0,)

In [19]:
print(df.columns.tolist())


['mean_0', 'mean_1', 'mean_2', 'mean_3', 'mean_4', 'mean_5', 'mean_6', 'mean_7', 'variance_0', 'variance_1', 'variance_2', 'variance_3', 'variance_4', 'variance_5', 'variance_6', 'variance_7', 'skewness_0', 'skewness_1', 'skewness_2', 'skewness_3', 'skewness_4', 'skewness_5', 'skewness_6', 'skewness_7', 'kurtosis_0', 'kurtosis_1', 'kurtosis_2', 'kurtosis_3', 'kurtosis_4', 'kurtosis_5', 'kurtosis_6', 'kurtosis_7', 'energy_0', 'energy_1', 'energy_2', 'energy_3', 'energy_4', 'energy_5', 'energy_6', 'energy_7', 'rms_0', 'rms_1', 'rms_2', 'rms_3', 'rms_4', 'rms_5', 'rms_6', 'rms_7', 'band_power_delta_0', 'band_power_delta_1', 'band_power_delta_2', 'band_power_delta_3', 'band_power_delta_4', 'band_power_delta_5', 'band_power_delta_6', 'band_power_delta_7', 'band_power_theta_0', 'band_power_theta_1', 'band_power_theta_2', 'band_power_theta_3', 'band_power_theta_4', 'band_power_theta_5', 'band_power_theta_6', 'band_power_theta_7', 'spectral_entropy_0', 'spectral_entropy_1', 'spectral_entropy_2

In [20]:
clip_index = df.columns.get_loc('clip')

df2 = df.iloc[:, :clip_index + 1]

df2.reset_index(drop=True, inplace=True)

In [21]:
ratings_file_path = r'Extracted_features\Dt_SelfReports.mat' 
ratings_data = scipy.io.loadmat(ratings_file_path)['Ratings']  # Shape: (5, 58, 36)

ratings_rows = []

for i in range(58):  # 58 participants
    for j in range(36):  # 36 clips
        ratings_row = {
            'Arousal': ratings_data[0, i, j].item(),  
            'Valence': ratings_data[1, i, j].item(),
            'Engagement': ratings_data[2, i, j].item(),
            'Liking': ratings_data[3, i, j].item(),
            'Familiarity': ratings_data[4, i, j].item()
        }
        ratings_rows.append(ratings_row) 

ratings_df = pd.DataFrame(ratings_rows)


df2 = pd.concat([df2, ratings_df], axis=1)

print(df2.columns.tolist())


['mean_0', 'mean_1', 'mean_2', 'mean_3', 'mean_4', 'mean_5', 'mean_6', 'mean_7', 'variance_0', 'variance_1', 'variance_2', 'variance_3', 'variance_4', 'variance_5', 'variance_6', 'variance_7', 'skewness_0', 'skewness_1', 'skewness_2', 'skewness_3', 'skewness_4', 'skewness_5', 'skewness_6', 'skewness_7', 'kurtosis_0', 'kurtosis_1', 'kurtosis_2', 'kurtosis_3', 'kurtosis_4', 'kurtosis_5', 'kurtosis_6', 'kurtosis_7', 'energy_0', 'energy_1', 'energy_2', 'energy_3', 'energy_4', 'energy_5', 'energy_6', 'energy_7', 'rms_0', 'rms_1', 'rms_2', 'rms_3', 'rms_4', 'rms_5', 'rms_6', 'rms_7', 'band_power_delta_0', 'band_power_delta_1', 'band_power_delta_2', 'band_power_delta_3', 'band_power_delta_4', 'band_power_delta_5', 'band_power_delta_6', 'band_power_delta_7', 'band_power_theta_0', 'band_power_theta_1', 'band_power_theta_2', 'band_power_theta_3', 'band_power_theta_4', 'band_power_theta_5', 'band_power_theta_6', 'band_power_theta_7', 'spectral_entropy_0', 'spectral_entropy_1', 'spectral_entropy_2

In [22]:
# threshold for allowed null values
threshold = len(df2.columns) / 2

rows_to_delete = df2[df2.isnull().sum(axis=1) > threshold].index

print("Rows being deleted due to excessive null values:", rows_to_delete.tolist())

df_cleaned = df2.drop(rows_to_delete)

df_cleaned.reset_index(drop=True, inplace=True)

# Check the cleaned DataFrame
print(df_cleaned.info())


Rows being deleted due to excessive null values: [9, 196, 204, 210, 219, 221, 222, 225, 230, 235, 237, 241, 246, 247, 250, 278, 288, 291, 292, 297, 302, 307, 312, 313, 316, 317, 322, 328, 332, 376, 384, 392, 423, 437, 439, 443, 448, 449, 452, 464, 470, 474, 484, 491, 493, 502, 504, 505, 507, 508, 509, 510, 511, 512, 513, 514, 515, 516, 517, 518, 519, 520, 521, 522, 523, 524, 525, 526, 527, 528, 529, 530, 531, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 548, 549, 550, 551, 552, 553, 554, 555, 556, 557, 558, 559, 560, 561, 562, 563, 564, 565, 566, 567, 568, 569, 570, 571, 572, 573, 574, 575, 576, 577, 578, 579, 580, 581, 582, 583, 584, 585, 586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 598, 599, 600, 601, 602, 603, 604, 605, 606, 607, 608, 609, 610, 611, 612, 615, 616, 617, 618, 620, 621, 624, 626, 627, 629, 630, 637, 639, 642, 645, 647, 1296, 1297, 1298, 1299, 1300, 1301, 1302, 1303, 1304, 1305, 1306, 1307, 1308, 1309, 1310, 1311, 1312, 

In [23]:
#here I needed to make each column carry +ve inputs

min_valence = df_cleaned['Valence'].min()
min_arousal = df_cleaned['Arousal'].min()
min_engagement = df_cleaned['Engagement'].min()
min_liking = df_cleaned['Liking'].min()
min_familiarity = df_cleaned['Familiarity'].min()


df_cleaned['Arousal'] += (abs(min_arousal)+1)
df_cleaned['Valence'] += (abs(min_valence)+1)
df_cleaned['Engagement'] += (abs(min_engagement)+1)
df_cleaned['Liking'] += (abs(min_liking)+1)
df_cleaned['Familiarity'] += (abs(min_familiarity)+1)

In [24]:
df_cleaned['stress'] = (
    df_cleaned['Arousal'] ** 0.5 *
    df_cleaned['Valence'] ** -0.4 *
    df_cleaned['Engagement'] ** 0.3 *
    df_cleaned['Liking'] ** -0.4 *
    df_cleaned['Familiarity'] ** -0.2
)

In [25]:
print(df_cleaned['stress'].describe())


count    1128.000000
mean        1.134312
std         0.599205
min         0.273941
25%         0.770876
50%         0.984703
75%         1.272366
max         4.287854
Name: stress, dtype: float64


In [26]:
#stress value threshold
df_cleaned['is_stressed'] = df_cleaned['stress'].apply(lambda x: 1 if x >= 1 else 0)



In [27]:
print(df_cleaned['is_stressed'].value_counts())


is_stressed
0    611
1    519
Name: count, dtype: int64


In [28]:

numeric_columns = df_cleaned.select_dtypes(include=['number']).columns
df_cleaned[numeric_columns] = df_cleaned[numeric_columns].fillna(df_cleaned[numeric_columns].mean())

nan_counts = df_cleaned.isna().sum()
print("NaN values per column after imputation:\n", nan_counts[nan_counts > 0])


NaN values per column after imputation:
 energy_0                22
energy_1               110
energy_2                42
energy_3                48
energy_4                84
energy_5                55
energy_6                98
energy_7                22
cross_correlation_0     10
cross_correlation_1     91
cross_correlation_2     23
cross_correlation_3     31
cross_correlation_4     66
cross_correlation_5     28
cross_correlation_6     79
cross_correlation_7     14
dtype: int64


In [29]:
for column in df_cleaned.columns:
    if df_cleaned[column].isna().sum() > 0:
        df_cleaned[column].fillna(df_cleaned[column].mean(), inplace=True)

nan_counts_final = df_cleaned.isna().sum()
print("NaN values per column after second imputation:\n", nan_counts_final[nan_counts_final > 0])


NaN values per column after second imputation:
 Series([], dtype: int64)


C:\Users\amols\AppData\Local\Temp\ipykernel_21884\306984826.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_cleaned[column].fillna(df_cleaned[column].mean(), inplace=True)


In [30]:
# Check for any remaining NaN values
nan_counts = df_cleaned.isna().sum()
print("NaN values per column after imputation:\n", nan_counts[nan_counts > 0])

NaN values per column after imputation:
 Series([], dtype: int64)


In [31]:
print(list(df_cleaned.columns))


['mean_0', 'mean_1', 'mean_2', 'mean_3', 'mean_4', 'mean_5', 'mean_6', 'mean_7', 'variance_0', 'variance_1', 'variance_2', 'variance_3', 'variance_4', 'variance_5', 'variance_6', 'variance_7', 'skewness_0', 'skewness_1', 'skewness_2', 'skewness_3', 'skewness_4', 'skewness_5', 'skewness_6', 'skewness_7', 'kurtosis_0', 'kurtosis_1', 'kurtosis_2', 'kurtosis_3', 'kurtosis_4', 'kurtosis_5', 'kurtosis_6', 'kurtosis_7', 'energy_0', 'energy_1', 'energy_2', 'energy_3', 'energy_4', 'energy_5', 'energy_6', 'energy_7', 'rms_0', 'rms_1', 'rms_2', 'rms_3', 'rms_4', 'rms_5', 'rms_6', 'rms_7', 'band_power_delta_0', 'band_power_delta_1', 'band_power_delta_2', 'band_power_delta_3', 'band_power_delta_4', 'band_power_delta_5', 'band_power_delta_6', 'band_power_delta_7', 'band_power_theta_0', 'band_power_theta_1', 'band_power_theta_2', 'band_power_theta_3', 'band_power_theta_4', 'band_power_theta_5', 'band_power_theta_6', 'band_power_theta_7', 'spectral_entropy_0', 'spectral_entropy_1', 'spectral_entropy_2

In [32]:

X = df_cleaned.drop(columns=['participant', 'clip', 'stress' ,'is_stressed'])
y = df_cleaned['is_stressed']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#normalising
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = Sequential([
    Dense(64, input_shape=(X_train.shape[1],), activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid') 
])

model.compile(optimizer=Adam(learning_rate=0.001), 
              loss=BinaryCrossentropy(),
              metrics=[BinaryAccuracy()])

history = model.fit(X_train, y_train, epochs=11, batch_size=32, validation_split=0.2, verbose=1)

test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


Epoch 1/11


c:\Users\amols\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - binary_accuracy: 0.5173 - loss: 0.6945 - val_binary_accuracy: 0.6740 - val_loss: 0.6385
Epoch 2/11
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.7122 - loss: 0.6012 - val_binary_accuracy: 0.7348 - val_loss: 0.5573
Epoch 3/11
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.8288 - loss: 0.4939 - val_binary_accuracy: 0.8122 - val_loss: 0.4497
Epoch 4/11
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.9074 - loss: 0.3368 - val_binary_accuracy: 0.8619 - val_loss: 0.3493
Epoch 5/11
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.9386 - loss: 0.2379 - val_binary_accuracy: 0.9116 - val_loss: 0.2814
Epoch 6/11
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.9536 - loss: 0.1466 - val_binary_accuracy: 0.9227 - val_loss: 0.2370
Epoch 7/11
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.9694 - loss: 0.1158 - val_binary_accuracy: 0.9392 - val_loss: 0.1986
Epoch 8/11
23/23 ━━━━━━━━━━━━━━

In [33]:
low_level_features = [col for col in df_cleaned.columns if 'mean' in col or 'variance' in col or 'kurtosis' in col or 'skewness' in col]
mid_level_features = [col for col in df_cleaned.columns if 'band_power' in col or 'energy' in col  or 'spectral_entropy' in col ]
high_level_features = [col for col in df_cleaned.columns if 'fractal_dimension' in col or 'hurst_exponent' in col or 'stress' in col]

y = df_cleaned['is_stressed']

X_low = df_cleaned[low_level_features]
X_mid = df_cleaned[mid_level_features]
X_high = df_cleaned[high_level_features]

X_train_low, X_test_low, y_train, y_test = train_test_split(X_low, y, test_size=0.2, random_state=42)
X_train_mid, X_test_mid, _, _ = train_test_split(X_mid, y, test_size=0.2, random_state=42)
X_train_high, X_test_high, _, _ = train_test_split(X_high, y, test_size=0.2, random_state=42)

scaler_low = StandardScaler()
scaler_mid = StandardScaler()
scaler_high = StandardScaler()

X_train_low = scaler_low.fit_transform(X_train_low)
X_test_low = scaler_low.transform(X_test_low)
X_train_mid = scaler_mid.fit_transform(X_train_mid)
X_test_mid = scaler_mid.transform(X_test_mid)
X_train_high = scaler_high.fit_transform(X_train_high)
X_test_high = scaler_high.transform(X_test_high)


low_input = Input(shape=(X_train_low.shape[1],))
low_dense = Dense(32, activation='relu')(low_input)
low_output = Dense(16, activation='relu')(low_dense)

mid_input = Input(shape=(X_train_mid.shape[1],))
mid_dense = Dense(32, activation='relu')(mid_input)
mid_output = Dense(16, activation='relu')(mid_dense)

concatenated_low_mid = Concatenate()([low_output, mid_output])

high_input = Input(shape=(X_train_high.shape[1],))
high_dense = Dense(16, activation='relu')(high_input)
high_output = Dense(8, activation='relu')(high_dense)

concatenated_all = Concatenate()([concatenated_low_mid, high_output])

final_output = Dense(1, activation='sigmoid')(concatenated_all)

final_model = Model(inputs=[low_input, mid_input, high_input], outputs=final_output)

final_model.compile(optimizer=Adam(learning_rate=0.001), 
                    loss=BinaryCrossentropy(),
                    metrics=[BinaryAccuracy()])

history = final_model.fit([X_train_low, X_train_mid, X_train_high], y_train, 
                          epochs=11, batch_size=32, validation_split=0.2, verbose=1)

test_loss, test_accuracy = final_model.evaluate([X_test_low, X_test_mid, X_test_high], y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


Epoch 1/11


c:\Users\amols\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_22', 'keras_tensor_25', 'keras_tensor_29']. Received: the structure of inputs=('*', '*', '*')
  warnings.warn(


23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - binary_accuracy: 0.4615 - loss: 0.7661 - val_binary_accuracy: 0.5359 - val_loss: 0.7337
Epoch 2/11
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.5418 - loss: 0.6996 - val_binary_accuracy: 0.6354 - val_loss: 0.6973
Epoch 3/11
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.6897 - loss: 0.6448 - val_binary_accuracy: 0.6740 - val_loss: 0.6662
Epoch 4/11
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.7636 - loss: 0.6109 - val_binary_accuracy: 0.7845 - val_loss: 0.6300
Epoch 5/11
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.8493 - loss: 0.5667 - val_binary_accuracy: 0.8398 - val_loss: 0.5843
Epoch 6/11
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.8980 - loss: 0.5323 - val_binary_accuracy: 0.8950 - val_loss: 0.5283
Epoch 7/11
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.9249 - loss: 0.4529 - val_binary_accuracy: 0.9282 - val_loss: 0.4584
Epoch 8/11
23/23 ━━━━━━━━━━━━━━